<a href="https://colab.research.google.com/github/kacperkurowski/CE888/blob/master/Assignment/Santander-Customer-Transaction-Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 1

In [3]:
pip install scikit-plot

import os
import seaborn as sns
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.model_selection import cross_val_score
import numpy as np

SyntaxError: ignored

In [4]:
os.environ['KAGGLE_USERNAME'] = "kacperkurowski" # username from the json file

os.environ['KAGGLE_KEY'] = "7c927dd985620db03df77b3804e71a95" # key from the json file

NameError: ignored

#Unzipping Function

In [0]:
def unzip(file):
  with ZipFile(file, 'r') as zip:
    zip.extractall()
    print('extracted: ',file)

##1.Loading Data

In [0]:
!kaggle competitions download -c santander-customer-transaction-prediction # api copied from kaggle

##1.Removing Files that are not used

In [0]:
!rm sample_submission.csv.zip
!ls

#1.Unzipping The Files

In [0]:
unzip("/content/train.csv.zip")

##Loading the files

In [0]:
df_train_Orig = pd.read_csv("train.csv", header = 0, sep = ',', quotechar = '"')

print(df_train_Orig)

##Plotting Histogram of Santander-Customer-Transaction-Prediction Train Data

In [0]:
y_df = df_train_Orig["var_0"].copy()
y = y_df.values
sns_plot = sns.distplot(y, kde=False, rug=True)

##Adding Test Columns for testing Covariate Shift

Here i incorporate covariate shift, by placing all the data that is higher than 9.0 into the train column and everything below 9.0 into the test. 
This will cause there to be a covariate shift, as the train and test data use different data.

In [0]:
target_var = "target"
id_var = "ID_code"


df_train = df_train_Orig.where(df_train_Orig["var_0"]>9.0).dropna().drop(columns=[target_var,id_var])
df_test = df_train_Orig.where(df_train_Orig["var_0"]<=9.0).dropna().drop(columns=[target_var,id_var])
df_train['is_test'] = 0
df_test['is_test'] = 1
print(df_train)
print(df_test)

I have implemented covariate shift in the 'var_0' variable by removing it into another array if it is smaller than 9.0. This will allow me to test for the present covariate shift later.

The test and train data is labelled with a 'is_test' variable so that after merging and training on the data is complete, the accuracy can be deduced (AUC score).

#Classifying to Obtain AUC 

In [0]:
def classify(MergedData, feature):
  X = MergedData.drop(columns=[feature]) 
  y = MergedData[feature]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
  
  rfc = RandomForestClassifier()
  rfc.fit(X_train, y_train)
  y_test_score = rfc.predict_proba(X_test)
  skplt.metrics.plot_roc_curve(y_test, y_test_score)
  plt.show()
  print('AUC score: ', round(roc_auc_score(y_true=y_test, y_score=y_test_score[:,1]), 4))
  return X,y;

Classified = pd.concat([df_train, df_test], axis = 0)
X,y = classify(Classified, "is_test")

The score is above approximately 0.75, signifying the there is covariate shift present in the data. This is done by testing if the testing data and training data can be differentiated from one another. A result os 1.0 means that the data is very easy to differentiate and therefore isnt similar.

#Searching for columns causing Covariate Shift

In [0]:
dropped_columns = []
model = RandomForestClassifier(n_estimators = 100, max_depth = 8,min_samples_leaf = 5)
for n in X.columns:
    score = cross_val_score(model,pd.DataFrame(X[i]),y,cv=2,scoring='roc_auc')
    if (np.mean(score) > 0.8):
        dropped_columns.append(n)
        print(n,np.mean(score))

print(dropped_columns)


Variables with covariate shift are detected, as removing them can influence the covariate shift of the data. (this is one method for doing this, however it also usually effect the data depending on how key the removed variables were).

#Removing Features Responsible for Covariate Shift

In [0]:
print(Classified.shape)

def histogram(df_test, df_train, data):
  print("Histogram of dataset on '",data,"'")
  histo = sns.distplot(df_test[[data]].values.T[0], color="blue", kde=False, label="Testing data")
  histo = sns.distplot(df_train[[data]].values.T[0], color="red", kde=False, label="Training data")
  legend = histo.legend()

df_orig = df_train_Orig.drop(columns=[target_var,id_var])
normalized_test=(df_test-df_orig.min())/(df_orig.max()-df_orig.min())
normalized_train=(df_train-df_orig.min())/(df_orig.max()-df_orig.min())

histogram(df_test, df_train,"var_0")

A Histogram of the Training and Testing data can be seen. The testing data is made from the training data as the result column wasnt present in both the training and testing data and it made it difficult to test the accuracy of the classification.

#Splitting and Classifying after the removal of covariate shift

In [0]:
unused,unused2 = classify(Classified,"is_test")

After the covariate shift was completely removed from the data it can be seen that the result of the classifier is approximately 0.5. This signifiies that the classifier coul;dnt differentiate between testing and training data, meaning that covariate shift was succesfully removed.

#Checking performance with covariate shift present

In [0]:
classify(df_train_Orig, "target")

#Checking performance after the removal of covariate shift

In [0]:
df_no_covariate = df_train_Orig.drop(columns=[id_var,"var_0"])
classify(df_no_covariate,"target")

In [0]:
##Let's use simple Random Forest as Classifier
#rfc = RandomForestClassifier()
#rfc.fit(X_train, y_train)
#y_test_score = rfc.predict_proba(X_test)

#import matplotlib.pyplot as plt

#skplt.metrics.plot_roc_curve(y_test, y_test_score)
#plt.show()


##AUC score is about 0.5
#print('AUC score: ', round(roc_auc_score(y_true=y_test, y_score=y_test_score[:,1]), 4))